from google.colab import drive
drive.mount("/content/gdrive/")
%cd /content/gdrive/My\ Drive/work_space/Chatbot/NLU/Mark1
%pip install import_ipynb
import import_ipynb

In [108]:
import re
import numpy as np
import pandas as pd
%pip install persian
%pip install hazm
import hazm
import persian
import os
import re
import pandas as pd
import DataSet
import numpy as np
from hazm import *
from nltk import ngrams
import DataSet
import NLP

In [0]:
def Start(lang,path):
  docs, entities ,slots,intent_ngram = DataSet.loader(lang=lang,path=path)
  docs,labels,classes,slots = NLP.prepare_dataset(docs,slots,intent_ngram)
  vectors = NLP.docs2vectors(docs=docs["text"])
  return dict(zip(["vectors","labels","classes","slots","entities"],[vectors,labels,classes,slots,entities]))

In [0]:
from nltk import ngrams

def find_match_entity(row ,**kywrd):
  #print(row["value"],len(word_tokenize(Normalizer().normalize(row["value"]))))
  words = word_tokenize(Normalizer().normalize(kywrd["text"]))
  ng_words = ngrams(words,len(word_tokenize(Normalizer().normalize(row["value"]))))
  sim= []
  for w in ng_words:
    sim.append(NLP.similarity(a=row["value"],b=" ".join(w))[0])
  res = (row["entity"],row["value"]) if np.max(sim) > 0.7 else 0
  
  return res

In [0]:
def __entities_extarctor(**kywrd):
  intent = kywrd['intent']
  text = kywrd["text"]
  slots =  kywrd["slots"]
  entities = kywrd["entities"]
  entities =entities.set_index("name")
  
  req_ent= list(set([x for sublist in slots[slots["intent"] == intent]["entities"] for x in sublist]))
  entity_keys= []
  entity_patterns= []
  for e in req_ent:
    if entities.loc[e]["type"] == "list":
      for ele in entities.loc[e]["occurrences"]:

        entity_keys.append({"entity":e,"value":ele["name"] })

        if len(ele["synonyms"]) != 0:
          entity_keys += [{"entity":e,"value":x } for x in ele["synonyms"]]
    else:
      entity_patterns.append((e,entities.loc[e]["pattern"]))
  
  ent_patt_mch = []
  for n,p in entity_patterns:
    res = re.findall(p,text)
    ent_patt_mch += list(zip([n]*len(res),res))

  entity_keys = pd.DataFrame(entity_keys)

  temp =entity_keys.apply(find_match_entity,axis=1,text = text)
  ents = list(temp[temp != 0]) + ent_patt_mch

  return ents

In [0]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
path = "./models"


def Train(save_model = False,load_model = True , **kywrd):
  trainX,trainy = kywrd["vectors"],kywrd["labels"]
  if load_model:
    clf = __load_classifier(path)
  else:
    clf = RandomForestClassifier(max_depth=12,max_features=50,n_estimators=20)
    clf.fit(trainX,trainy)
  if save_model:
    __save_model(clf,path)
  print("train Score :", clf.score(trainX,trainy))
  kywrd["classifier"] =clf
  return kywrd

#classifier = engine_train(vectors = tf_idf , labels= labels)

In [0]:
import pickle
import datetime
def __save_model(classifier,path):
  try:
    cls_name =str(type(classifier))
    cls_name = cls_name[cls_name.rfind(".")+1:-2]
    with open(os.path.join(path,'{0}{1}.cpkl'.format(cls_name, str(datetime.datetime.now())[5:16])), 'wb') as f:
      pickle.dump(classifier, f)
    return True
  except err:
    raise ValueError(err)

import glob
def __load_classifier(path):
  
  for i,sp in enumerate(glob.glob(os.path.join(path,"*.cpkl"))):
    print(i,")",sp)
  select = input("select model that you want load or 'exit':")
  if select == "exit":
    return 0
  try:
    path = glob.glob(os.path.join(path,"*.cpkl"))[int(select)]
    with open(path, 'rb') as f:
      classifier = pickle.load(f)
    return classifier
  except err:
    raise ValueError(err)
    

In [0]:
# intent,ents,slots = Predict("میخوام درس هوش محاسباتی و اخلاق اسلامی را اخذ کنم 95125856 و هوش مصنوعی را حذف کنم .",**ready_to_predict)
# text ="میخوام درس هوش محاسباتی و اخلاق اسلامی را اخذ کنم 95125856 و هوش مصنوعی را حذف کنم ."


def __to_ngram(row,**kywrd):
  text = kywrd["text"]
  slots = kywrd["slots"]
  
  if row["val"] in text:
    text = text.replace(row["val"],"٪٪٪٪")
  else:
    for wrong in ngrams(word_tokenize(text),len(word_tokenize(row["val"]))):
      if NLP.similarity(row["val"]," ".join(wrong))[0] > 0.65:
        text = text.replace(" ".join(wrong),"٪٪٪٪")
        break
      

  for word in row["another_keys"]:
    text = text.replace(word, "")
  
  grams = ngrams(word_tokenize(NLP.normalize(text)[np.random.randint(0,2)]),3)
  text_grams = []
  for g in grams:
    if "٪٪٪٪" in g[:2]:
      text_grams.append(" ".join(g))

  res = []
  for tg in text_grams:
    for sg in list(slots[slots["id"] == row["slot"] ]["ngrams"])[0]:
      res.append(NLP.similarity(tg,sg))

  row["similarity"] = np.average(res)
  row["name"] = slots[slots["id"] == row["slot"]]["name"].iloc[0]
  return row


def __slots_extractor(ents,slots,text,intent):

  slt = slots[slots["intent"] == intent]
  slots_per_entity = pd.DataFrame(columns=["val","another_keys","slot"])
  for n_ent , v_ent in ents:
    for i,slot in slt.iterrows():
      if n_ent in slot["entities"] and slot["intent"] == intent:
        slots_per_entity = slots_per_entity.append(pd.Series({"val":v_ent,"another_keys":[v for n , v in ents if v !=v_ent ] ,"slot":slot["id"]}),ignore_index=True)

  found_slot_df = slots_per_entity.apply(__to_ngram,text=text,slots=slots,axis=1)
  result = []
  for val,sim in found_slot_df.groupby("val"):
    idx = np.argmax(sim["similarity"])
    result.append((val,sim["name"].iloc[idx]))
  
  return result


#slots[slots["id"] == "gkv0BbM1njQKhwmsudG5G" ]["name"][0]




In [0]:
def Predict(text , **kywrd):
  vec = NLP.query2vector(text)
  intent =kywrd["classes"][int(kywrd["classifier"].predict(vec))]
  ents = __entities_extarctor(intent= intent , text = text ,**kywrd)
  found_slots = __slots_extractor(ents,kywrd["slots"],text,intent)
  return intent,ents,found_slots



In [0]:
def ShowResult(result):
  intent,ents,found_slots =  result
  print()
  print("Intent : ","="*40)
  print("\t",intent)

  print("\nEntities : \n","="*40)
  [print("\t",x," : ",y) for y,x in ents]
  print("\nSlots    : \n","="*40)
  [print("\t",x," : ",y) for y,x in found_slots]